# Imports

In [1]:
#from src import Build_mc,Upload_geof,Upload_geof_geografica,Upload_litologia,plot_filtered_values,plot_raw_mag_data,plot_raw_gama_data,plot_histograms,plot_interpolated_histograms,describe_geologico, plotBoxplots,remove_negative_values,sintetic_grid,traditional_interpolation,plot_interpolated_histograms,pop_nodata
from src import *
from pylab import *

from tqdm import tqdm
from shapely.ops import transform
from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import verde as vd
import numpy as np
import os
import pyproj
import verde_source as vds

import seaborn as sns
import xarray as xr
import matplotlib

from sklearn_som.som import SOM
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib widget
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
'''def sintetic_grider(quadricula=None,p_size=None):
    ids = list(quadricula.keys())
    wgs84=pyproj.CRS('EPSG:4326')
    for id in tqdm(ids):
        folha = quadricula[id]['folha']
        utm = pyproj.CRS('EPSG:'+folha['EPSG'])
        carta_geografica = folha['geometry']
        project = pyproj.Transformer.from_crs(wgs84,utm, always_xy=True).transform
        carta_utm = transform(project,carta_geografica)
        break
'''

"def sintetic_grider(quadricula=None,p_size=None):\n    ids = list(quadricula.keys())\n    wgs84=pyproj.CRS('EPSG:4326')\n    for id in tqdm(ids):\n        folha = quadricula[id]['folha']\n        utm = pyproj.CRS('EPSG:'+folha['EPSG'])\n        carta_geografica = folha['geometry']\n        project = pyproj.Transformer.from_crs(wgs84,utm, always_xy=True).transform\n        carta_utm = transform(project,carta_geografica)\n        break\n"

## Construindo Quadrícula

In [4]:
quadricula = Build_mc(escala='100k',ID=['SF23_VD','SF23_VC','SF23_YA','SF23_YB'],verbose=True)

  0%|                                                                                                                    | 0/4 [00:00<?, ?it/s]


AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [ ]:
list_ids = list(quadricula.keys())
#print(list_ids)

'''
for id in list_ids:
    print(f' - Folha: {id}')
    carta=quadricula[id]
    data_list = list(carta.keys())
    print(data_list)
    for data in data_list[4:]:
        print(f'    - {data}')
        print(f'        - {list(quadricula[id][data].columns)}')
'''    
def print_quadriculas(quadricula=quadricula):
    list_ids = list(quadricula.keys())
    for id in list_ids:
        print(f' - Folha {id}')
        carta = quadricula[id]
        list_carta_keys = list(carta.keys())
        print(f'    - {list_carta_keys}:')
        for key in list_carta_keys:
            print(carta[key])
            print('')

In [ ]:
print_quadriculas(quadricula)

### Construindo grid regular sintético

_Verde gridder_

In [ ]:
from verde import grid_coordinates

In [ ]:
def batch_grid_coordinates(quadricula,spacing=0.001,pixel_register=True):
    list_id = list(quadricula.keys())
    
    #print(f' Folhas disponíveis: {list_id}')
    for id in list_id:
        folha = quadricula[id]
        print(f' Folha  {id}')

        df = quadricula[id]['folha']
        area = (df['geometry'].bounds[0],df['geometry'].bounds[2],
                df['geometry'].bounds[1],df['geometry'].bounds[3])
        
        xu,yu = grid_coordinates(region=area,spacing=spacing,pixel_register=pixel_register)
       
        df['coords'] = xu,yu
        df['area'] = area
        x={'area':df}
        quadricula[id].update(x)
        

batch_grid_coordinates(quadricula)

In [ ]:
print_quadriculas(quadricula)

In [ ]:
quadricula['SF23_YA_V']['area']['coords']

In [ ]:
quadricula['SF23_YA_V']['folha']['coords']

## Adicionando dados brutos à Quadrícula

In [ ]:
#Upload_geof(quadricula,'gama_line_1105','mag_line_1105',1000)
#gama_1039,mag_1039=Upload_geof(quadricula,'gama_1039','mag_1039',1000)

Upload_geof_geografica(quadricula,'gama_line_1105','mag_line_1105',0.01)

In [ ]:
Upload_geof_geografica(quadricula,'gama_line_1089','mag_line_1089')

In [ ]:
quadricula['SF23_YA_III']['gama_line_1105'].head()

In [ ]:
geof_list_ids = list(quadricula.keys())
for id in geof_list_ids:
    print(f' - Folha: {id}')
    carta=quadricula[id]
    data_list = list(carta.keys())
    print(data_list)
    for data in data_list[4:]:
        print(f'    - {data}')
        print(f'        - {list(quadricula[id][data].columns)}')
    print('')


In [ ]:
quadricula=pop_nodata(quadricula)
len(quadricula.keys())

In [ ]:
for id in list(quadricula.keys()):
    print(f' - Folha:  {id}')
    carta = quadricula[id]
    print(f'    - {list(carta.keys())}')
    print('')

## Vizualisando Área de Cobertura

In [ ]:
plt.figure(figsize=(9,6))
for id in list(quadricula.keys()):
    carta=quadricula[id]['area']
    plt.plot(*(carta['geometry']).exterior.xy,color='black')
    plt.axis('scaled')
    plt.scatter(carta['coords'][0],carta['coords'][1],s=1)    
plt.suptitle('Área de cobertura dos levantamentos aerogeofísicos')
plt.axis('scaled')
plt.tight_layout()

In [ ]:
list(carta.keys())

In [ ]:
plt.figure(figsize=(14,9))
for id in list(quadricula.keys()):
    carta=quadricula[id]
    #print(carta.keys())
    plt.plot(*(carta['area']['geometry_proj']).exterior.xy,color='black')
    plt.axis('scaled')
    for data in list(carta.keys())[:]:
        if 'gama' in data:
            #print(data)
            plt.scatter(carta[data].X,carta[data].Y,c=carta[data].UTHRAZAO,cmap='hsv',s=1,marker='H')    
        else:
            None
plt.suptitle('Área de cobertura dos levantamentos aerogeofísicos')
plt.axis('scaled')
plt.tight_layout()

#### Visualizando dados Radiométricos Brutos

In [ ]:
df = quadricula['SF23_YA_III']['gama_line_1105']
#remove_negative_values()

In [ ]:
plot_histograms(df,500,'Levantamento 1105 Dados radiométricos brutos')

# Interpolação dos dados Brutos

## Verde Splines

## Método Linear

In [ ]:
list(quadricula['SF23_YA_III']['area'].keys())

In [ ]:
quadricula['SF23_YA_III']['area']['area']

In [ ]:
traditional_interpolation(quadricula,'mag_line_1105','gama_line_1105','linear','geof_1105',extrapolate=True,projec='geog')
#traditional_interpolation(quadricula,'mag_1039','gama_1039','linear','geof_1039',extrapolate=True)
#traditional_interpolation(quadricula,'mag_3022','gama_3022','linear','geof_3022')

In [ ]:
ds_c = quadricula['SF23_YA_III']['geof_1105_linear']
#ds_d = quadricula['SF23_VD']['geof_1105_linear']
#ds_a = quadricula['SF23_YA']['geof_1105_linear']
#ds_b = quadricula['SF23_YB']['geof_1105_linear']


In [ ]:
ds_c

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(12,9))
list_data = list(ds_c.keys())
for i,ax in enumerate(axes.flatten()):
    
    if i < len(list_data):
        if list_data[i] == 'GMT':
            ax.imshow(ds_a[list_data[i]],cmap='bwr')
            ax.set(title=ds_a[list_data[i]].name)
        elif list_data[i]=='MDT':
            ax.imshow(ds_a[list_data[i]],cmap='terrain')
            ax.set(title=ds_a[list_data[i]].name)
        
        else:
            ax.imshow(ds_a[list_data[i]])
            ax.set(title=ds_a[list_data[i]].name)

In [ ]:
fig, axes = plt.subplots(3,3,figsize=(14,12))
list_data = list(ds_a.keys())
for i,ax in enumerate(axes.flatten()):
    
    if i < len(list_data):
        if list_data[i] == 'GMT':
            ax.imshow(ds_a[list_data[i]],cmap='bwr')
            ax.set(title=ds_a[list_data[i]].name)
        elif list_data[i]=='MDT':
            ax.imshow(ds_a[list_data[i]],cmap='terrain')
            ax.set(title=ds_a[list_data[i]].name)
        
        else:
            ax.imshow(ds_a[list_data[i]])
            ax.set(title=ds_a[list_data[i]].name)

# Classificações Não-Supervisionadas

## Rotulando amostras com classes litológicas

In [ ]:
'''
import shapely.speedups
from shapely import geometry
shapely.speedups.enable()

#geof_1089_linear['geometry'] = [geometry.Point(x,y) for x, y in zip(geof_1089_linear['X'], geof_1089_linear['Y'])]
#gdf_1089_linear = geof_1089_linear.set_geometry('geometry')

#gdf_1089_linear.set_crs('EPSG:32723',inplace=True)
#gdf_1089_linear.geometry

#Upload_litologia(quadricula,'litologia_100k')

litologia=quadricula['SB24_ZB_II']['litologia_100k']
litologia.to_crs('EPSG:32724',inplace=True)
print(litologia.crs)
litologia.reset_index(drop=True,inplace=True)

dic_litologico = describe_geologico(litologia)
print(litologia.columns)

#print(dic_litologico['SIGLA']['len'])
#print(dic_litologico['SIGLA']['lista'])

#litologia.plot('SIGLA',figsize=(16,16),legend=True)

#geof_1089_linear['closest_unit'] = geof_1089_linear['geometry'].apply(lambda x: litologia['SIGLA'].iloc[litologia.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

#geof_1089_linear.to_csv('/home/ggrl/database/csv/SB24_ZB_II_gama_linear_100m.csv',index=False)
'''

## Self-organizing maps (SOM)

### Pixel size

In [ ]:
#help(pcolormesh)

In [ ]:
print(list(quadricula.keys()))
print('')
print(quadricula['SF23_YA'].keys())
print('')
ds_a = quadricula['SF23_YA']['geof_1105_linear']
ds_b = quadricula['SF23_YB']['geof_1105_linear']

In [ ]:
ds_b.LONGITUDE.values.max()

In [ ]:
ds_a.LONGITUDE.values.max()

In [ ]:
ds_b

In [ ]:
ds_b['LATITUDE']

In [ ]:
ds_a['LATITUDE']

In [ ]:
ds = xr.combine_by_coords([ds_a,ds_b])

In [ ]:
#for id in tqdm(list(quadricula.keys())):
#    ds = quadricula[id]['geof_1105_linear']
features = list(ds.keys())
print(features)

In [ ]:
df = ds[features].to_dataframe()
df = df.reset_index()
df

In [ ]:
xpixel_size = (ds.LONGITUDE.max().values-ds.LONGITUDE.min().values)/ds.LONGITUDE.size
ypixel_size = (ds.LATITUDE.max().values-ds.LATITUDE.min().values)/ds.LATITUDE.size
print('x:', xpixel_size, 'y:', ypixel_size)

In [ ]:
nx = ds.LATITUDE.size
ny = ds.LONGITUDE.size

print(ny,nx,ny/nx)

xs = df.LONGITUDE.values.reshape(nx,ny)
ys = df.LATITUDE.values.reshape(nx,ny)

In [ ]:
def plot_corr(dataframe, size=10):
    plt.figure(figsize = (size+size*0.2, size), facecolor='w')
    corrMatrix = dataframe.corr()
    sns.heatmap(np.round(corrMatrix,2), annot=True,vmin=-1,vmax=1,cmap='bwr',center=0)
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0, ha='right')

plot_corr(df, size=12)
#plt.savefig('figs/correlation_matrix.png', dpi=400, bbox_inches='tight')

In [ ]:
scaler = StandardScaler()
data = scaler.fit_transform(df[features].values)
data.shape

In [ ]:
n_clusters=4
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=50)
lito_SOM.fit(data)

In [ ]:
# predição de classes
predictions = lito_SOM.predict(data)

In [ ]:
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']

In [ ]:
# classes weights
fig, ax = plt.subplots(figsize=(19,19))
im=ax.matshow(lito_SOM.weights,cmap='bwr',vmin=-1.5,vmax=1.5)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(10, 10)
plt.show()

In [ ]:
id_ = [1,2,3,4]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)

In [ ]:
idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)

In [ ]:
print(xs.shape,ys.shape)

In [ ]:
fig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data (SF23_YB_I)')
plt.axis('scaled')
plt.show()

In [ ]:
features = ['MDT','GMT','F_factor']

scaler = StandardScaler()
data = scaler.fit_transform(df[features].values)
n_clusters=10
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=10000)
lito_SOM.fit(data)
# predição de classes
predictions = lito_SOM.predict(data)
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']


# classes weights
fig, ax = plt.subplots(figsize=(19,19))
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(10, 10)
plt.show()
import matplotlib
id_ = [1,2,3,4,5,6,7,8,9,10]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)
idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)
fig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data ( Aguas de Lindóia - SF23_YA_III4)')
plt.axis('scaled')
plt.show()

In [ ]:
features = ['MDT','GMT','F_factor','CTCOR']

scaler = StandardScaler()
data = scaler.fit_transform(df[features].values)
n_clusters=10
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=10000)
lito_SOM.fit(data)
# predição de classes
predictions = lito_SOM.predict(data)
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']


# classes weights
fig, ax = plt.subplots(figsize=(19,19))
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(10, 10)
plt.show()
import matplotlib
id_ = [1,2,3,4,5,6,7,8,9,10]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)
idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)
fig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data ( Aguas de Lindóia - SF23_YA_III4)')
plt.axis('scaled')
plt.show()

In [ ]:
# get attributes
features = list(SF23_YA_III_4.keys())
print(features)
# transform a xarray.Dataset into pandas.Dataframe
df = SF23_YA_III_4[features].to_dataframe()
df = df.reset_index()

In [ ]:
# visualize correlations
plot_corr(df, size=12)
# SELECT FEATURES
features = ['MDT','GMT','F_factor']
print(f' - Selected Features: {features}')

In [ ]:
# get pixel size
xpixel_size = (ds.LONGITUDE.max().values-ds.LONGITUDE.min().values)/ds.LONGITUDE.size
ypixel_size = (ds.LATITUDE.max().values-ds.LATITUDE.min().values)/ds.LATITUDE.size
print('x:', xpixel_size, 'y:', ypixel_size)
# get number of pixels
nx = ds.LATITUDE.size
ny = ds.LONGITUDE.size
print(f'x:{ny} ; y:{nx} | x/y: {ny/nx}')
# reshape numpy.array into tuple of arrays
xs = df.LONGITUDE.values.reshape(nx,ny)
ys = df.LATITUDE.values.reshape(nx,ny)
# transform - fit dataframe to scallar matrix of shape 250 x 250
scaler = StandardScaler()
data = scaler.fit_transform(df[features].values)
print(f' - Scalar shape:   {data.shape}')
# Clusters, n, sigma, iterations
n_clusters=10
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=10000)
lito_SOM.fit(data)
# predição de classes
predictions = lito_SOM.predict(data)
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']
# classes weights
fig, ax = plt.subplots(figsize=(19,19))
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')
# 
plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(10, 10)
plt.show()
import matplotlib
id_ = [1,2,3,4,5,6,7,8,9,10]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)
idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)
fig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
# 
plt.suptitle('SOM - Aerogeophysical Data ( Aguas de Lindóia - SF23_YA_III4)')
plt.axis('scaled')
plt.show()